In [22]:
import spacy
import random
from spacy.util import minibatch, compounding

In [69]:
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


# training data
TRAIN_DATA = [
    ("Horses are too tall and they pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("Do they bite?", {
        'entities': []
    }),

    ("horses are too tall and they pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("horses pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("they pretend to care about your feelings, those horses", {
        'entities': [(48, 54, 'ANIMAL')]
    }),

    ("horses?", {
        'entities': [(0, 6, 'ANIMAL')]
    })
]
def main(model='models', output_dir='models', n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    print("creating pipeline")
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    print("adding labels")
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    print("get names of other pipes")
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        print("training a new model")
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

main()

Loaded model 'models'
creating pipeline
adding labels
get names of other pipes
training a new model
Losses {'ner': 4.807638990743499}
Losses {'ner': 3.4844751060009003}
Losses {'ner': 2.545009471476078}
Losses {'ner': 3.929951224476099}
Losses {'ner': 3.560788057744503}
Losses {'ner': 3.98904275149107}
Losses {'ner': 4.016806233674288}
Losses {'ner': 2.817154921591282}
Losses {'ner': 2.288880430161953}
Losses {'ner': 2.941999234266719}
Losses {'ner': 1.6972038322419394}
Losses {'ner': 1.7995849028229713}
Losses {'ner': 1.2077336050570011}
Losses {'ner': 2.0141837115747068}
Losses {'ner': 2.7251112684607506}
Losses {'ner': 2.3611611500382423}
Losses {'ner': 1.3638695368540852}
Losses {'ner': 1.8124835267663002}
Losses {'ner': 2.5978066101670265}
Losses {'ner': 1.6401193737983704}
Losses {'ner': 2.6338713839650154}
Losses {'ner': 1.7399880141019821}
Losses {'ner': 2.1653596349060535}
Losses {'ner': 1.8020861595869064}
Losses {'ner': 2.154963593930047}
Losses {'ner': 1.0924126191011965}
L

# Test the trained model

In [70]:
nlp_simple = spacy.load('models')

In [71]:
doc = nlp_simple('I have heard Alex just bough a new horse for his sister.')

In [72]:
for ents in doc.ents:
    print(ents)

In [43]:
dir(a)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'check_flag',
 'cluster',
 'flags',
 'from_bytes',
 'has_vector',
 'is_alpha',
 'is_ascii',
 'is_bracket',
 'is_currency',
 'is_digit',
 'is_left_punct',
 'is_lower',
 'is_oov',
 'is_punct',
 'is_quote',
 'is_right_punct',
 'is_space',
 'is_stop',
 'is_title',
 'is_upper',
 'lang',
 'lang_',
 'like_email',
 'like_num',
 'like_url',
 'lower',
 'lower_',
 'norm',
 'norm_',
 'orth',
 'orth_',
 'prefix',
 'prefix_',
 'prob',
 'rank',
 'sentiment',
 'set_attrs',
 'set_flag',
 'shape',
 'shape_',
 'similarity',
 'suffix',
 'suffix_',
 'text',
 'to_bytes',
 'vector',
 'vector_norm',
 'vocab']